# 데모

## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from matplotlib import pyplot as plt
from matplotlib import rcParams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import seaborn as sns
import warnings

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [4]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 학습데이터 로드

In [5]:
data_dir = Path('../data/dacon-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

In [6]:
algo_name = 'RF'
feature_name = 'tfidf'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [7]:
trn = pd.read_csv(trn_file, index_col=0)
tst = pd.read_csv(tst_file, index_col=0)

## NLTK 예시

In [8]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer

In [9]:
s = trn.text[4]
print(s)

“Have mercy, gentlemen!” odin flung up his hands. “Don’t write that, anyway; have some shame. Here I’ve torn my heart asunder before you, and you seize the opportunity and are fingering the wounds in both halves.... Oh, my God!”


In [10]:
tokens = word_tokenize(s)
print(tokens)

['“', 'Have', 'mercy', ',', 'gentlemen', '!', '”', 'odin', 'flung', 'up', 'his', 'hands', '.', '“', 'Don', '’', 't', 'write', 'that', ',', 'anyway', ';', 'have', 'some', 'shame', '.', 'Here', 'I', '’', 've', 'torn', 'my', 'heart', 'asunder', 'before', 'you', ',', 'and', 'you', 'seize', 'the', 'opportunity', 'and', 'are', 'fingering', 'the', 'wounds', 'in', 'both', 'halves', '....', 'Oh', ',', 'my', 'God', '!', '”']


## Bag-of-Words 피처 생성

In [11]:
vec = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
y = trn.author.values

## 로지스틱회귀 모델 학습

In [12]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [13]:
Xx = X
Yy = y

## 하이퍼 파라미터 튜닝

In [14]:
X_trn, X_val, y_trn, y_val = train_test_split(Xx, Yy, test_size=.25, random_state=seed)

In [15]:
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin, pyll
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from pathlib import Path
import warnings

In [16]:
params = {
    'n_jobs'                    : -1,
    'random_state'              : seed,
    'verbose'                   : 1,
    'warm_start'                : False,  #False로 해야 비교가 되지...
    
    
    #'bootstrap'                 : True,
    #'oob_score'                 : hp.choice('oob_score', [True, False]),
    'bootstrap'                : False,
    
    #class_weight               : None
    
    #'min_weight_fraction_leaf' : .0
    #'min_samples_split'        : 2
    # 'max_leaf_nodes'          : None,
    # 'min_impurity_decrease'   : .0,     : 불순도(?)감소가 주어진 값 이상이 되어야 분리됨,
    # 'min_impurity_split'      : None,      : early-stopping을 위한 threshold
    #'ccp_alpha'                : .0,
    #'max_samples'              : None,
}

#oob_score는 bootstrap이 true일 때만 가능
space = {
    'criterion'                 : hp.choice('criterion', ['entropy', 'gini']),
    'max_features'              : hp.choice('max_features', ['sqrt', 'log2', None]), 
    'n_estimators'              : hp.choice('n_estimators', np.arange(50, 300, dtype=int)),
    #overfitting 방지
    'min_samples_leaf'          : hp.choice('min_samples_leaf', np.arange(1, 30, dtype=int)),
    'max_depth'                 : hp.quniform('max_depth', 10, 100, 10)
}

In [17]:
# from sklearn.ensemble import RandomForestClassifier

# #튜닝 코드: 이를 통해 구한 최적화된 파라미터는 아래 제시되어 있습니다.
# def objective(hyperparams):
#     model = RandomForestClassifier(**params, **hyperparams)
#     model.fit(X=X_trn, y=y_trn)
#     score = accuracy_score(y_val, model.predict(X_val))

#     return {'loss': -score, 'status': STATUS_OK, 'model': model}

# trials = Trials()
# best = fmin(fn=objective, space=space, trials=trials,
#             algo=tpe.suggest, max_evals=30, verbose=0)
# best

In [18]:
tuned = {
    'criterion': 'gini',
    'max_depth': 80.0,
    'max_features': 'log2',
    'min_samples_leaf': 1,
    'n_estimators': 139
}

In [20]:
from sklearn.ensemble import RandomForestClassifier

p = np.zeros((X.shape[0], n_class))
p_tst = np.zeros((X_tst.shape[0], n_class))
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
    clf = RandomForestClassifier(**params, **tuned)

    clf.fit(X[i_trn], y[i_trn])
    p[i_val, :] = clf.predict_proba(X[i_val])
    p_tst += clf.predict_proba(X_tst) / n_class

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 139 out of 139 | elapsed:    4.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 139 out of 139 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 139 out of 139 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 139 out of 139 | elapsed:    4.2s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0

In [22]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Accuracy (CV):  65.5314%
Log Loss (CV):   1.1879


In [23]:
np.savetxt(p_val_file, p, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

## 제출 파일 생성

In [24]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub

(19617, 5)


,0,1,2,3,4
index,,,,,
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
19612,0,0,0,0,0
19613,0,0,0,0,0
19614,0,0,0,0,0


In [25]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2,3,4
index,,,,,
0,0.2082,0.2624,0.2546,0.1918,0.0830
1,0.1854,0.3116,0.1326,0.2127,0.1576
2,0.3082,0.1491,0.1679,0.2499,0.1250
3,0.2009,0.0827,0.3480,0.1665,0.2019
4,0.3374,0.1630,0.1672,0.2177,0.1147


In [26]:
sub.to_csv(sub_file)